In [1]:
import pandas as pd
proj_list= pd.read_csv('Scenario3_InputData.csv', index_col=['ID'])

In [2]:
proj_list

,Programme,Type,Location,Region,CAPEX,NPV,CAPEX Yr1,CAPEX Yr2,CAPEX Yr3
ID,,,,,,,,,
1,Piping,AssetIntegrity,Facility J,Zone4,5,6.00,4,1,0
2,Instrumentation,AssetIntegrity,Facility F,Zone1,10,11.00,1,5,4
3,Mechanical,AssetIntegrity,Facility C,Zone3,10,14.00,2,4,4
4,Debottlenecking,Growth,Facility D,Zone4,5,7.00,5,0,0
5,Piping,AssetIntegrity,Facility C,Zone3,10,12.00,4,4,2
6,Expansion,Growth,Facility C,Zone3,15,20.00,7,8,0
7,Revamp,Growth,Facility F,Zone1,15,16.00,7,7,1
8,Piping,AssetIntegrity,Facility D,Zone4,2,5.00,1,1,0
9,Revamp at Facility E Start Yr1,Growth,Facility E,Zone5,3,4.00,2,1,0


In [3]:
proj_list.dtypes

Programme     object
Type          object
Location      object
Region        object
CAPEX          int64
NPV          float64
CAPEX Yr1      int64
CAPEX Yr2      int64
CAPEX Yr3      int64
dtype: object

In [4]:
capexconstraints=[20,25,15] 

In [5]:
import pulp

# Create A Model

phasing = pulp.LpProblem("Maximise", pulp.LpMaximize)

Selection = pulp.LpVariable.dicts("Selection", proj_list.index, cat='Binary')

In [6]:
# Set The Objective Function                                                                                                         
phasing += pulp.lpSum(Selection[idx]*proj_list.loc[idx]["NPV"] for idx in proj_list.index)


In [9]:
# Set The Constraints 

phasing += sum([Selection[idx] * proj_list.loc[idx]["CAPEX Yr1"] for idx in proj_list.index]) <= capexconstraints[0]
phasing += sum([Selection[idx] * proj_list.loc[idx]["CAPEX Yr2"] for idx in proj_list.index]) <= capexconstraints[1]
phasing += sum([Selection[idx] * proj_list.loc[idx]["CAPEX Yr3"] for idx in proj_list.index]) <= capexconstraints[2]

phasing += Selection[5] + Selection[11] == 1 #Either Project 5 or 10 must be chosen
phasing += Selection[4] == Selection[8] #Both Projects 4 and 8 must go together
phasing += Selection[9] + Selection[10] == 1 #Either Project 9a or 9b must be chosen
phasing += Selection[7] <= Selection[2] #Project 7 is optional choice as long as Project 2 is selected


In [10]:
# Run The Solver(s)

%time phasing.solve() #equivalent to phasing.solve(pulp.PULP_CBC_CMD()) as CBC is PulP's default solver

pulp.LpStatus[phasing.status]

# Print our objective function value and Output Solution
print (pulp.value(phasing.objective))


Wall time: 1.05 s
72.75


In [11]:
#Convert output into user friendly output for viewing or downloading 
pulpsolution=pd.DataFrame(columns = ["Selection Y/N","CAPEX Yr1 Selected","CAPEX Yr2 Selected", "CAPEX Yr3 Selected", "NPV Selected"])
for idx in proj_list.index:
    to_append = [Selection[idx].value(), Selection[idx].value()*proj_list.loc[idx]["CAPEX Yr1"],Selection[idx].value()*proj_list.loc[idx]["CAPEX Yr2"],Selection[idx].value()*proj_list.loc[idx]["CAPEX Yr3"],Selection[idx].value()*proj_list.loc[idx]["NPV"]]
    perprojectyear = pd.Series(to_append, index = pulpsolution.columns)
    pulpsolution = pulpsolution.append(perprojectyear, ignore_index=True)

pulpsolution.index += 1 
pulpsolution

pulpoutput = pd.concat([proj_list, pulpsolution], axis=1)
pulpoutput

,Programme,Type,Location,Region,CAPEX,NPV,CAPEX Yr1,CAPEX Yr2,CAPEX Yr3,Selection Y/N,CAPEX Yr1 Selected,CAPEX Yr2 Selected,CAPEX Yr3 Selected,NPV Selected
1,Piping,AssetIntegrity,Facility J,Zone4,5,6.00,4,1,0,0.0,0.0,0.0,0.0,0.00
2,Instrumentation,AssetIntegrity,Facility F,Zone1,10,11.00,1,5,4,1.0,1.0,5.0,4.0,11.00
3,Mechanical,AssetIntegrity,Facility C,Zone3,10,14.00,2,4,4,1.0,2.0,4.0,4.0,14.00
4,Debottlenecking,Growth,Facility D,Zone4,5,7.00,5,0,0,1.0,5.0,0.0,0.0,7.00
5,Piping,AssetIntegrity,Facility C,Zone3,10,12.00,4,4,2,1.0,4.0,4.0,2.0,12.00
6,Expansion,Growth,Facility C,Zone3,15,20.00,7,8,0,1.0,7.0,8.0,0.0,20.00
7,Revamp,Growth,Facility F,Zone1,15,16.00,7,7,1,0.0,0.0,0.0,0.0,0.00
8,Piping,AssetIntegrity,Facility D,Zone4,2,5.00,1,1,0,1.0,1.0,1.0,0.0,5.00
9,Revamp at Facility E Start Yr1,Growth,Facility E,Zone5,3,4.00,2,1,0,0.0,0.0,0.0,0.0,0.00
10,Revamp at Facility E Start Yr2,Growth,Facility E,Zone5,3,3.75,0,2,1,1.0,0.0,2.0,1.0,3.75


In [12]:
CAPEX_Totals=[pulpsolution['CAPEX Yr1 Selected'].sum(),pulpsolution['CAPEX Yr2 Selected'].sum(),pulpsolution['CAPEX Yr3 Selected'].sum()]

CAPEX_CheckSum= pd.DataFrame()
CAPEX_CheckSum['Year'] = [1,2,3]
CAPEX_CheckSum['CAPEX Phasing'] = CAPEX_Totals
CAPEX_CheckSum['Constraints'] = capexconstraints
print('Total NPV=', round(pulpsolution['NPV Selected'].sum(),2))
print()
print(CAPEX_CheckSum.to_string(index = False))

Total NPV= 72.75

 Year  CAPEX Phasing  Constraints
    1           20.0           20
    2           24.0           25
    3           11.0           15


In [13]:
pulpoutput['Selection Y/N']

1     0.0
2     1.0
3     1.0
4     1.0
5     1.0
6     1.0
7     0.0
8     1.0
9     0.0
10    1.0
11    0.0
Name: Selection Y/N, dtype: float64